In [100]:
import pandas as pd
import numpy as np
import re

In [92]:
data = pd.read_csv('../data/combined_data_v3.csv')

In [93]:
data.sample(n = 10)

,source,headline,year,date,is_sarcastic
256657,ABC Australia,hospital piano proves to be a joy for staff pa...,2019,2019-03-16,0
268920,ABC Australia,wests tigers robbie farah grateful to make 300...,2019,2019-07-23,0
187282,ABC Australia,amnesty accuses is us led coalition of mosul w...,2017,2017-07-12,0
191460,ABC Australia,sinkhole florida retiree jokes about living ju...,2017,2017-08-10,0
247522,ABC Australia,walkley awards winners,2018,2018-12-05,0
343507,atlantic,The Age of Reverse Censorship,2018,2018-06-26,0
192460,ABC Australia,all blacks security guard has public nuisance ...,2017,2017-08-18,0
52613,Huffington Post,This American Muslim Is Actually Seeking Out P...,2017,2017-06-30,0
126573,ABC Australia,townsville bats helicoptered away,2016,2016-05-27,0
371076,onion,Colleges Send Out Reminder To Graduates That 2...,2018,2018-06-07,1


### Preprocessing Steps
1. Standardize common abbreviations* (WIP)
    1. u.s. --> usa
1. Lowercase
1. Expand Contractions
1. Optional:
    1. Remove Source Specific Language
    1. Remove Profanity
1. Remove Special Characters


In [94]:
def replace_words(text, replace_dict = translate_dict):
    tokens = []
    for w in text.split():
        word = w
        for t in replace_dict.keys():
            if w == t:
                word = replace_dict[t]
        tokens.append(word)
    
    return " ".join(tokens)

In [95]:
data['clean'] = data['headline']

## Standardize Common Abbreviations
translate_dict = {
    "US": "USA",
    "U.S.": "USA",
    "u.s.": "USA",
    "u.s": "USA",
}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, translate_dict))

In [96]:
# Lowercase
data['clean'] = data['clean'].apply(lambda x: x.lower())

In [97]:
# Expand Contractions
contractions_dict = { "ain't": "are not","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who's": "who is",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, contractions_dict))


In [98]:
# Example should read: who is afraid
data['clean'][77627]

'who is afraid to think?'

In [101]:
## Remove Special Characters
data['clean'] = data['clean'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))

In [102]:
data['clean']

0         former versace store clerk sues over secret bl...
1         the roseanne revival catches up to our thorny ...
2         jk rowling wishes snape happy birthday in the ...
3                                advancing the worlds women
4             the fascinating case for eating labgrown meat
                                ...                        
385165    sarah palin wants afghans banned from americas...
385166                breaking bristol palin struck by semi
385167    obama to send large shipment of thoughts and p...
385168    trump university offers sarah palin honorary d...
385169    sarah palin granted hysterical landmark design...
Name: clean, Length: 385170, dtype: object

In [104]:
data.to_csv('../data/combined_clean.csv', index = False, sep = "|")

SyntaxError: invalid syntax (<ipython-input-104-54f3b58f73a3>, line 1)